In [ ]:
import numpy as np
import sklearn as sk
from sklearn import ensemble
from sklearn.preprocessing import StandardScaler, RobustScaler, MinMaxScaler
import tensorflow as tf
tfk=tf.keras
import matplotlib.pyplot as plt
from corner import corner

### Plot the learning process of the Neural Network

In [ ]:
plt.figure(figsize=(8,6))
plt.plot(history.history['loss'],label='train')
plt.plot(history.history['val_loss'],label='val')
plt.ylim([0.002,0.006])
plt.legend()

### Evaluate the Neural Network

Make predictions on the test dataset and plot them against the true values.

In [ ]:
pred = model.predict(X_test)

if norm_y:
    pred   = Y_scaler.inverse_transform(pred)

R2 = [sk.metrics.r2_score(Y_test[:,i], pred[:,i]) for i in range(len(Y_test[0]))]


fig, axs = plt.subplots( 2, 3, figsize=(18,12) )

c=0
for i in range(2):
    for j in range(3):
        if c==num_targets:
            break
            
        axs[i,j].scatter( Y_test[:,c], pred[:,c], alpha=.03, label = r'$R^2 = $' + str( round(R2[c], 2) ) )
        axs[i,j].plot( [-13,3000], [-13,3000], 'r--' )
        
        axs[i,j].set_xlim( [min(Y_test[:,c]), max(Y_test[:,c])] )
        axs[i,j].set_ylim( [min(Y_test[:,c]), max(Y_test[:,c])] )
        
        axs[i,j].set_xlabel('True ' + names[c])
        axs[i,j].set_ylabel('Predicted ' + names[c])
        axs[i,j].grid(True)
        axs[i,j].legend()
        c+=1

axs[1,2].remove()
plt.suptitle( r'$R^2=$ ' + str( round( np.mean(R2), 2 ) ) )
plt.show()

### WASP-12b retrieval

Draw 100 samples of WASP-12b's spectrum scattered according to the errorbars of the observation.

In [ ]:
N_samples = 10000

trans = WASP12b[:,1].reshape(1,13)
noise = WASP12b[:,2].reshape(1,13)

trans = np.repeat(trans, N_samples, axis=0)
noise = np.repeat(noise, N_samples, axis=0)

samples = trans + np.random.normal(scale=noise)

plt.errorbar(0.01*WASP12b[:,0], trans[0], yerr=noise[0], fmt='k.-', capsize=3)
plt.xlabel(r'$\lambda$ ($\mu m$)')
plt.ylabel(r'$(R_P/R_S)^2$ (%)')
plt.show()

Get the predictions of the Neural Network.

In [ ]:
if norm_x:
    samples = X_scaler.transform(samples)

w12b_pred = model.predict(samples)

if norm_y:
    w12b_pred = Y_scaler.inverse_transform(w12b_pred)

Plot the predictions in a corner plot. This shows us the posterior probability distribution of the parameters. Blue lines from Nested Sampling retrieval in Márquez-Neila et al (2018).

In [ ]:
CORNER_KWARGS = dict(
    smooth=.75,
    label_kwargs=dict(fontsize=16),
    title_kwargs=dict(fontsize=16),
    quantiles=[0.16, 0.5, 0.84],
    show_titles=True,
    truths=[1105, -3, -8.5, -8.4, -2.8],
    range=[(500,3000),(-13,0),(-13,0),(-13,0),(-13,3)]
)

fig=corner(w12b_pred, labels=names, **CORNER_KWARGS)